In [1]:
import requests
from datasets import load_dataset, Dataset
import re
import pyarrow.parquet as pq
from tqdm import tqdm

# Load the dataset using the traditional method
dataset = load_dataset("allenai/c4", "az")

def count_sentences(text):
    sentences = re.findall(r'[.!?]', text)
    return len(sentences)

sentence_count = 0
for split in ["train", "validation"]:
    for i in tqdm(dataset[split], desc=f"Processing {split}"):
        sentence_count += count_sentences(i["text"])

print("Sentence count with traditional load method: ", sentence_count)

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/256 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/34 [00:00<?, ?it/s]

Processing validation: 100%|█████████████| 5239/5239 [00:00<00:00, 18803.00it/s]


Sentence count with traditional load method:  146936849


KeyboardInterrupt: 

In [4]:
dataset['train']

Dataset({
    features: ['text', 'timestamp', 'url'],
    num_rows: 5285720
})

In [5]:
dataset['validation']

Dataset({
    features: ['text', 'timestamp', 'url'],
    num_rows: 5239
})

In [7]:
import pandas as pd
# Convert the dataset splits to DataFrames
train_df = pd.DataFrame(dataset['train'])
validation_df = pd.DataFrame(dataset['validation'])

# Merge the DataFrames
merged_df = pd.concat([train_df, validation_df], ignore_index=True)

In [8]:
merged_df

,text,timestamp,url
0,Mühacirət mövzusunun öyrənilməsi ictimai-siyas...,2019-01-23 08:22:09,https://azertag.az/xeber/Muhaciret_movzusunun_...
1,"﻿ Lurer.com | Յուրի Խաչատուրովն ասել է, որ չի ...",2019-04-25 08:32:04,https://lurer.com/?p=295680&l=am
2,Xiaomi Mi 8 Lite 128 GB Siyah Dist Dual Sim Ce...,2019-09-22 20:17:05,https://www.istanbulbilisim.com/xiaomi-mi-8-li...
3,Palitra qəzetiSərnişin avtobuslarını çirkləndi...,2019-09-19 06:37:08,http://www.palitranews.az/news.php?id=65577
4,Times.am | Բաց և թափանցիկ համացանցային հարթակ\...,2017-06-28 20:54:07,http://blog.times.am/?p=203219&l=am
...,...,...,...
5290954,Tahir Mirkişili sahibkarlarla onlayn görüş keç...,2020-08-03 15:30:19,https://mail.168saat.com/1464-tahir-mirkisili-...
5290955,Qadınları sevməyi öyrənin - Yazılı şəkillər » ...,2017-04-24 19:20:52,http://qadinla.com/heyat-terzi/yazili-sekiller...
5290956,"Ölkədə dəlixanaların sayı artmalıdır, ya yox?\...",2020-01-27 21:24:56,https://sputnik.az/news/20191117/422316616/Psi...
5290957,"Azərbaycan prezidenti: Bu faşizmdir, irqçilikd...",2018-11-12 23:02:27,http://payiz.az/news/4766/azerbaycan-prezident...


In [9]:
df = _

In [14]:
import pandas as pd
from tqdm import tqdm

# Assuming merged_df is the DataFrame that you have after merging 'train' and 'validation' datasets

# Function to count words in a text
def count_words(text):
    words = text.split()
    return len(words)

# Calculate the total word count in the merged DataFrame
total_word_count = 0
for text in tqdm(merged_df['text'], desc="Counting words in merged dataset"):
    total_word_count += count_words(text)

print("Total word count in merged dataset: ", total_word_count)

Counting words in merged dataset: 100%|█| 5290959/5290959 [01:59<00:00, 44212.72

Total word count in merged dataset:  1747607602


In [10]:
import re
import pandas as pd
from tqdm import tqdm
import string

# Azerbaijani characters
az_characters = "A-Za-zÇçƏəĞğİıÖöŞşÜü"
digits = "0-9"
punctuation = string.punctuation

# Compile a regex pattern to match only Azerbaijani letters, digits, punctuation, and whitespace
pattern = re.compile(f"^[{az_characters}{digits}{re.escape(punctuation)}\s]+$")

# Function to check if the text matches the Azerbaijani character set, digits, and punctuation
def is_azerbaijani(text):
    return bool(pattern.match(text))

# Assuming `merged_df` is the merged DataFrame from the previous steps
# Filter the DataFrame
filtered_df = merged_df[merged_df['text'].apply(is_azerbaijani)]
filtered_df

,text,timestamp,url
2,Xiaomi Mi 8 Lite 128 GB Siyah Dist Dual Sim Ce...,2019-09-22 20:17:05,https://www.istanbulbilisim.com/xiaomi-mi-8-li...
6,Qadin saatlari 2018 - ən ucuz qiyməti və satış...,2019-10-18 14:33:56,https://alsatmall.com/tag/qadin-saatlari-2018
9,Elan nömrəsi: 406873\nOfisə kadr təiəb olunur:...,2020-08-09 15:06:48,https://unvan.az/ofise-xanim-kadr-teleb-olunur...
29,"Garmin Araç Navigasyonu Montaj Kiti Fiyatı, Ga...",2019-06-20 04:53:17,https://www.garminsaat.com/garmin-arac-navigas...
30,"2017, 24 Yanvar, çərşənbə axşamı, Bakı vaxtı 0...",2017-01-23 22:56:29,https://www.azadliq.org/a/24795291.html
...,...,...,...
5290929,"""Bir milyondan çox qeyri-leqal məşğulluq rəsmi...",2020-01-19 08:45:07,http://demokrat.az/news/6532
5290932,"Brifinq.az APA-ya istinadən xəbər verir ki, bu...",2017-01-23 16:54:42,http://brifinq.az/news/sience/36104-sagirdleri...
5290934,49 il danışa bilməyən azərbaycanlı qadının dil...,2019-05-27 07:49:23,https://www.medicina.az/49-il-danisa-bilmeyen-...
5290935,"Axar.az xəbər verir ki, bu, keçən ilin dövlət ...",2018-03-23 18:23:48,http://axar.az/news/iqtisadiyyat/227014.html


In [11]:
# Count sentences in the filtered DataFrame
def count_sentences(text):
    sentences = re.findall(r'[.!?]', text)
    return len(sentences)

sentence_count = 0
for text in tqdm(filtered_df['text'], desc="Processing filtered dataset"):
    sentence_count += count_sentences(text)

print("Sentence count in filtered dataset: ", sentence_count)

Processing filtered dataset: 100%|█| 1045725/1045725 [00:10<00:00, 96537.01it/s]

Sentence count in filtered dataset:  14360168


In [12]:
df = filtered_df
df = df.dropna(subset='text').drop_duplicates(subset='text')
df

,text,timestamp,url
2,Xiaomi Mi 8 Lite 128 GB Siyah Dist Dual Sim Ce...,2019-09-22 20:17:05,https://www.istanbulbilisim.com/xiaomi-mi-8-li...
6,Qadin saatlari 2018 - ən ucuz qiyməti və satış...,2019-10-18 14:33:56,https://alsatmall.com/tag/qadin-saatlari-2018
9,Elan nömrəsi: 406873\nOfisə kadr təiəb olunur:...,2020-08-09 15:06:48,https://unvan.az/ofise-xanim-kadr-teleb-olunur...
29,"Garmin Araç Navigasyonu Montaj Kiti Fiyatı, Ga...",2019-06-20 04:53:17,https://www.garminsaat.com/garmin-arac-navigas...
30,"2017, 24 Yanvar, çərşənbə axşamı, Bakı vaxtı 0...",2017-01-23 22:56:29,https://www.azadliq.org/a/24795291.html
...,...,...,...
5290929,"""Bir milyondan çox qeyri-leqal məşğulluq rəsmi...",2020-01-19 08:45:07,http://demokrat.az/news/6532
5290932,"Brifinq.az APA-ya istinadən xəbər verir ki, bu...",2017-01-23 16:54:42,http://brifinq.az/news/sience/36104-sagirdleri...
5290934,49 il danışa bilməyən azərbaycanlı qadının dil...,2019-05-27 07:49:23,https://www.medicina.az/49-il-danisa-bilmeyen-...
5290935,"Axar.az xəbər verir ki, bu, keçən ilin dövlət ...",2018-03-23 18:23:48,http://axar.az/news/iqtisadiyyat/227014.html


In [13]:
df.to_csv('c4-az-clean.csv')

In [16]:
import pandas as pd
from tqdm import tqdm

# Assuming merged_df is the DataFrame that you have after merging 'train' and 'validation' datasets

# Function to count words in a text
def count_words(text):
    words = text.split()
    return len(words)

# Apply the word count function to the 'text' column and create a new column 'word_count'
tqdm.pandas(desc="Calculating word counts")
df['word_count'] = df['text'].progress_apply(count_words)

# Calculate the total word count
total_word_count = df['word_count'].sum()

print("Total word count:", total_word_count)


Calculating word counts: 100%|█████| 1045725/1045725 [00:14<00:00, 73295.27it/s]

Total word count: 192723366


In [18]:
from collections import Counter

# Split the text into individual words and count their occurrences
word_counts = Counter(" ".join(df['text']).split())

# Print the count of different words
print("Count of different words:", len(word_counts))


Count of different words: 6238154
